# Complex computing environments
Now let's imagine some non-trivial tasks that require specific environments in which to compute 

In [ ]:
import os
import sys

import numpy as np

from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.decorators import on_cluster, mpi_task
from jobqueue_features.mpi_wrapper import mpi_wrap, which
from jobqueue_features.functions import set_default_cluster

Let's have 2 different architecture types in this example.

In [ ]:
HSW = True
GPU = True

In [ ]:
set_default_cluster(CustomSLURMCluster)

For our software environment on the nodes, we need to leverage `env_extra`

In [ ]:
if GPU:
    GROMACS_gpu_cluster = CustomSLURMCluster(
        name="GROMACS_gpu_cluster",
        walltime="00:05:00",
        queue = "develgpus",
        nodes=2,
        mpi_mode=True,
        fork_mpi=True,
        queue_type="gpus",
        maximum_scale=5,
        env_extra=[
            "module --force purge",
            "module use /usr/local/software/jureca/OtherStages",
            "module load Stages/Devel-2019a",
            "module load Intel",
            "module load ParaStationMPI",
            "module load GROMACS",
            "module load GPUtil",  # Only required for our hello_world2.py example
            "module load dask",
            "module load jobqueue_features",
        ],
    )

if HSW:
    GROMACS_cluster = CustomSLURMCluster(
        name="GROMACS_cluster",
        walltime="00:05:00",
        queue = "devel",
        nodes=2,
        mpi_mode=True,
        fork_mpi=True,
        maximum_scale=5,
        env_extra=[
            "module --force purge",
            "module use /usr/local/software/jureca/OtherStages",
            "module load Stages/Devel-2019a",
            "module load Intel",
            "module load ParaStationMPI",
            "module load GROMACS",
            "module load dask",
            "module load jobqueue_features",
        ],
    )


In [ ]:
if GPU:

    @on_cluster(cluster=GROMACS_gpu_cluster, cluster_id="GROMACS_gpu_cluster", scale=6)
    @mpi_task(cluster_id="GROMACS_gpu_cluster")
    def run_mpi_gpu(**kwargs):
        print(which("gmx"))
        script_path = os.path.join(
            os.getenv("JOBQUEUE_FEATURES_EXAMPLES"), "resources", "helloworld2.py"
        )
        t = mpi_wrap(
            pre_launcher_opts='time -f "%e"',
            executable="python",
            exec_args=script_path,
            **kwargs
        )
        return t


if HSW:

    @on_cluster(cluster=GROMACS_cluster, cluster_id="GROMACS_cluster")
    @mpi_task(cluster_id="GROMACS_cluster")
    def run_mpi(**kwargs):
        print(which("gmx"))
        script_path = os.path.join(
            os.getenv("JOBQUEUE_FEATURES_EXAMPLES"), "resources", "helloworld2.py"
        )
        t = mpi_wrap(
            pre_launcher_opts='time -f "%e"',
            executable="python",
            exec_args=script_path,
            **kwargs
        )
        return t

In [ ]:
def complex_example():
    t_gpu = []
    t = []

    n_samples = 10
    for x in range(n_samples):
        if GPU:
            t_gpu.append(run_mpi_gpu())
        if HSW:
            t.append(run_mpi())

    if GPU:
        runtimes_gpu = [float((i.result()["err"]).split(b"\n")[-2]) for i in t_gpu]
        print("Example command:\n{}".format(t_gpu[0].result()["cmd"]))
        print("Example output:\n{}".format(t_gpu[0].result()["out"]))
        print(
            "GPU Compute Total (",
            len(runtimes_gpu),
            " samples) ",
            sum(runtimes_gpu),
            " : Average ",
            np.mean(runtimes_gpu),
            " +/- ",
            np.var(runtimes_gpu),
        )
    if HSW:
        runtimes = [float((i.result()["err"]).split(b"\n")[-2]) for i in t]
        print("Example command:\n{}".format(t[0].result()["cmd"]))
        print("Example output:\n{}".format(t[0].result()["out"]))
        print(
            "Cluster Compute Total (",
            len(runtimes),
            " samples)",
            sum(runtimes),
            " : Average ",
            np.mean(runtimes),
            " +/- ",
            np.var(runtimes),
        )

In [ ]:
%%time
complex_example()

But where is the output of
```python
print(which("gmx"))
```
in `run_mpi`/`run_mpi_gpu`?